# Modeling and solving search problems

Consider the following river crossing puzzle:

## Jealous Husbands Problem with an Island

There are n couples: $(𝐻_1,𝑊_1)$, $(𝐻_2,𝑊_2)$, …, $(𝐻_𝑛,𝑊_𝑛)$  where $𝐻_𝑖$ is husband $𝑖$ and $𝑊_𝑖$ is wife $𝑖$.

They need to cross from the left bank of a river to the right bank.

There is a boat that can carry at most two people at a time.

Important: There is an island midway between the left and right banks. The boat can stop at the island.

Rules:

1. Jealousy constraint: No wife may ever be in the presence of another husband (on any shore or the island or in the boat) unless her own husband is also present. Otherwise, fights (and failures) occur.

2. The boat needs at least one person to operate (no remote-controlled trips).

3. The boat can travel between:

            Left bank ↔ Island

            Island ↔ Right bank

4. All people, including the boat, start at the left bank.

Goal: get everyone (husbands and wives) safely to the right bank, obeying all the rules.


## Tasks:

1. Put the problem in a form of a search tree: root, nodes, solution and rules for constructing the childrens.
2. Solve it usinh a classic DFS
3. Solve it using a classic Back-tracking.
4. Solve it using the A* method.  

In [13]:
# model
from itertools import combinations
from copy import deepcopy

n = 3
people = [f"H{i+1}" for i in range(n)] + [f"W{i+1}" for i in range(n)]

initial_state = {
    "left": set(people),
    "island": set(),
    "right": set(),
    "boat": "left"
}

print(initial_state)


def is_goal(state):
    return len(state["right"]) == len(people) and state["boat"] == "right"

def valid_group(group):
    husbands = {p for p in group if p.startswith("H")}
    wives = {p for p in group if p.startswith("W")}
    for w in wives:
        wife_index = w[1]
        h = f"H{wife_index}"
        if h not in group:
            if any(hx != h for hx in husbands):
                return False
    return True

def is_valid(state):
    return all(valid_group(state[loc]) for loc in ["left", "island", "right"])

def get_successors(state):
    loc = state["boat"]

    people_here = state[loc]
    moves = list(combinations(people_here, 1)) + list(combinations(people_here, 2))

    next_states = []

    if loc == "island":
        for dest in ["left", "right"]:
            for move in moves:
                if not move: continue

                new_state = deepcopy(state)

                for p in move:
                    new_state[loc].remove(p)
                    new_state[dest].add(p)

                new_state["boat"] = dest
                
                if is_valid(new_state):
                    next_states.append(new_state)
    else:
        for move in moves:
            if not move: continue

            new_state = deepcopy(state)
            
            for p in move:
                new_state[loc].remove(p)
                new_state["island"].add(p)

            new_state["boat"] = "island"
            
            if is_valid(new_state):
                next_states.append(new_state)

    return next_states

def state_key(state):
    return (
        tuple(sorted(state["left"])),
        tuple(sorted(state["island"])),
        tuple(sorted(state["right"])),
        state["boat"]
    )

def dfs(state, visited, path):
    if is_goal(state):
        return path + [state]
    
    visited.add(state_key(state))
    
    for next_state in get_successors(state):
        key = state_key(next_state)
        
        if key not in visited:
            result = dfs(next_state, visited, path + [state])
            
            if result:
                return result

    return None


solution_path = dfs(initial_state, set(), [])

if solution_path:
    for i, s in enumerate(solution_path):
        print(f"Step {i}:")
        print(f"  Left:   {sorted(s['left'])}")
        print(f"  Island: {sorted(s['island'])}")
        print(f"  Right:  {sorted(s['right'])}")
        print(f"  Boat:   {s['boat']}")
        print()
else:
    print("No solution found.")



{'left': {'W3', 'H3', 'H1', 'H2', 'W2', 'W1'}, 'island': set(), 'right': set(), 'boat': 'left'}
Step 0:
  Left:   ['H1', 'H2', 'H3', 'W1', 'W2', 'W3']
  Island: []
  Right:  []
  Boat:   left

Step 1:
  Left:   ['H1', 'H2', 'W1', 'W2']
  Island: ['H3', 'W3']
  Right:  []
  Boat:   island

Step 2:
  Left:   ['H1', 'H2', 'H3', 'W1', 'W2']
  Island: ['W3']
  Right:  []
  Boat:   left

Step 3:
  Left:   ['H1', 'H2', 'H3', 'W1']
  Island: ['W2', 'W3']
  Right:  []
  Boat:   island

Step 4:
  Left:   ['H1', 'H2', 'H3', 'W1', 'W3']
  Island: ['W2']
  Right:  []
  Boat:   left

Step 5:
  Left:   ['H1', 'H3', 'W1', 'W3']
  Island: ['H2', 'W2']
  Right:  []
  Boat:   island

Step 6:
  Left:   ['H1', 'H3', 'W1', 'W3']
  Island: []
  Right:  ['H2', 'W2']
  Boat:   right

Step 7:
  Left:   ['H1', 'H3', 'W1', 'W3']
  Island: ['H2']
  Right:  ['W2']
  Boat:   island

Step 8:
  Left:   ['H1', 'H2', 'H3', 'W1', 'W3']
  Island: []
  Right:  ['W2']
  Boat:   left

Step 9:
  Left:   ['H1', 'H2', 'H3', 'W1

In [16]:
from itertools import combinations
from copy import deepcopy

n = 3
people = [f"H{i+1}" for i in range(n)] + [f"W{i+1}" for i in range(n)]

initial_state = {
    "left": set(people),
    "island": set(),
    "right": set(),
    "boat": "left"
}

def is_goal(state):
    return len(state["right"]) == len(people) and state["boat"] == "right"

def valid_group(group):
    husbands = {p for p in group if p.startswith("H")}
    wives = {p for p in group if p.startswith("W")}
    for w in wives:
        wife_index = w[1]
        h = f"H{wife_index}"
        if h not in group:
            if any(hx != h for hx in husbands):
                return False
    return True

def is_valid(state):
    return all(valid_group(state[loc]) for loc in ["left", "island", "right"])

def get_successors(state):
    loc = state["boat"]

    people_here = state[loc]
    moves = list(combinations(people_here, 1)) + list(combinations(people_here, 2))

    next_states = []
    for dest in ["left", "right"]:
        for move in moves:
            if not move:
                continue

            new_state = deepcopy(state)

            for p in move:
                new_state[loc].remove(p)
                new_state[dest].add(p)

            new_state["boat"] = dest

            if is_valid(new_state):
                next_states.append(new_state)

    return next_states

def state_key(state):
    return (
        tuple(sorted(state["left"])),
        tuple(sorted(state["island"])),
        tuple(sorted(state["right"])),
        state["boat"]
    )

def backtrack(state, visited, path):
    if is_goal(state):
        return path + [state]

    visited.add(state_key(state))

    for next_state in get_successors(state):
        key = state_key(next_state)

        if key not in visited:
            result = backtrack(next_state, visited, path + [state])

            if result:
                return result

    visited.remove(state_key(state))
    return None

solution_path = backtrack(initial_state, set(), [])

if solution_path:
    for i, s in enumerate(solution_path):
        print(f"Step {i}:")
        print(f"  Left:   {sorted(s['left'])}")
        print(f"  Island: {sorted(s['island'])}")
        print(f"  Right:  {sorted(s['right'])}")
        print(f"  Boat:   {s['boat']}")
        print()
else:
    print("No solution found.")


Step 0:
  Left:   ['H1', 'H2', 'H3', 'W1', 'W2', 'W3']
  Island: []
  Right:  []
  Boat:   left

Step 1:
  Left:   ['H1', 'H2', 'W1', 'W2']
  Island: []
  Right:  ['H3', 'W3']
  Boat:   right

Step 2:
  Left:   ['H1', 'H2', 'H3', 'W1', 'W2']
  Island: []
  Right:  ['W3']
  Boat:   left

Step 3:
  Left:   ['H1', 'H2', 'H3', 'W1']
  Island: []
  Right:  ['W2', 'W3']
  Boat:   right

Step 4:
  Left:   ['H1', 'H2', 'H3', 'W1', 'W3']
  Island: []
  Right:  ['W2']
  Boat:   left

Step 5:
  Left:   ['H1', 'H2', 'H3', 'W3']
  Island: []
  Right:  ['W1', 'W2']
  Boat:   right

Step 6:
  Left:   ['H1', 'H2', 'H3', 'W2', 'W3']
  Island: []
  Right:  ['W1']
  Boat:   left

Step 7:
  Left:   ['H1', 'H2', 'H3']
  Island: []
  Right:  ['W1', 'W2', 'W3']
  Boat:   right

Step 8:
  Left:   ['H1', 'H2', 'H3', 'W3']
  Island: []
  Right:  ['W1', 'W2']
  Boat:   left

Step 9:
  Left:   ['H3', 'W3']
  Island: []
  Right:  ['H1', 'H2', 'W1', 'W2']
  Boat:   right

Step 10:
  Left:   ['H2', 'H3', 'W2', 'W3']

In [ ]:
from itertools import combinations
from copy import deepcopy
import itertools
import heapq

n = 3
people = [f"H{i+1}" for i in range(n)] + [f"W{i+1}" for i in range(n)]

initial_state = {
    "left": set(people),
    "island": set(),
    "right": set(),
    "boat": ("left",)
}

def is_goal(state):
    return len(state["right"]) == len(people) and state["boat"][0] == "right"

def valid_group(group):
    husbands = {p for p in group if p.startswith("H")}
    wives = {p for p in group if p.startswith("W")}
    for w in wives:
        wife_index = w[1]
        h = f"H{wife_index}"
        if h not in group:
            if any(hx != h for hx in husbands):
                return False
    return True

def is_valid(state):
    return all(valid_group(state[loc]) for loc in ["left", "island", "right"])

def get_successors(state):
    loc = state["boat"][0]
    destinations = []
    
    if loc == "left":
        destinations = ["island"]
    elif loc == "island":
        destinations = ["left", "right"]
    else:
        destinations = ["island"]

    people_here = state[loc]
    moves = list(combinations(people_here, 1)) + list(combinations(people_here, 2))

    next_states = []
    for dest in destinations:
        for move in moves:
            if not move:
                continue
            new_state = deepcopy(state)
            for p in move:
                new_state[loc].remove(p)
                new_state[dest].add(p)
            new_state["boat"] = (dest,)
            if is_valid(new_state):
                next_states.append(new_state)

    return next_states

def state_key(state):
    return (
        tuple(sorted(state["left"])),
        tuple(sorted(state["island"])),
        tuple(sorted(state["right"])),
        state["boat"][0]
    )

def heuristic(state):
    remaining = len(state["left"]) + len(state["island"])
    return (remaining + 1) // 2


def a_star(start_state):
    open_set = []
    counter = itertools.count()
    heapq.heappush(open_set, (heuristic(start_state), 0, next(counter), start_state, []))
    visited = {}

    while open_set:
        f, g, _, current_state, path = heapq.heappop(open_set)
        key = state_key(current_state)

        if key in visited and visited[key] <= g:
            continue
        visited[key] = g

        if is_goal(current_state):
            return path + [current_state]

        for next_state in get_successors(current_state):
            new_g = g + 1
            new_f = new_g + heuristic(next_state)
            heapq.heappush(open_set, (new_f, new_g, next(counter), next_state, path + [current_state]))

    return None


solution_path = a_star(initial_state)

if solution_path:
    for i, s in enumerate(solution_path):
        print(f"Step {i}:")
        print(f"  Left:   {sorted(s['left'])}")
        print(f"  Island: {sorted(s['island'])}")
        print(f"  Right:  {sorted(s['right'])}")
        print(f"  Boat:   {s['boat'][0]}")
        print()
else:
    print("No solution found.")


Step 0:
  Left:   ['H1', 'H2', 'H3', 'W1', 'W2', 'W3']
  Island: []
  Right:  []
  Boat:   left

Step 1:
  Left:   ['H1', 'H2', 'W1', 'W2']
  Island: ['H3', 'W3']
  Right:  []
  Boat:   island

Step 2:
  Left:   ['H1', 'H2', 'H3', 'W1', 'W2']
  Island: ['W3']
  Right:  []
  Boat:   left

Step 3:
  Left:   ['H1', 'H2', 'H3']
  Island: ['W1', 'W2', 'W3']
  Right:  []
  Boat:   island

Step 4:
  Left:   ['H1', 'H2', 'H3', 'W3']
  Island: ['W1', 'W2']
  Right:  []
  Boat:   left

Step 5:
  Left:   ['H3', 'W3']
  Island: ['H1', 'H2', 'W1', 'W2']
  Right:  []
  Boat:   island

Step 6:
  Left:   ['H3', 'W3']
  Island: ['H1', 'W1']
  Right:  ['H2', 'W2']
  Boat:   right

Step 7:
  Left:   ['H3', 'W3']
  Island: ['H1', 'H2', 'W1']
  Right:  ['W2']
  Boat:   island

Step 8:
  Left:   ['H3', 'W3']
  Island: ['W1']
  Right:  ['H1', 'H2', 'W2']
  Boat:   right

Step 9:
  Left:   ['H3', 'W3']
  Island: ['H1', 'W1']
  Right:  ['H2', 'W2']
  Boat:   island

Step 10:
  Left:   ['H1', 'H3', 'W3']
  Isla